In [0]:
import shutil
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib

plt.style.use('ggplot')

In [0]:
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

get_available_gpus()

In [0]:
print(1)

## Difference Between Tensor and Python Variable

In [0]:
var = 1.
print("python variable: ", var)
print()

var_tensor = tf.constant(1.)
print("tensor: ", var_tensor)

In [0]:
with tf.Session() as sess:
    print(sess.run(var_tensor))

<br/>
<br/>
<br/>
## Simple Regression Example
- Formula: $ Y = 0.5X + 0.3 $

In [0]:
formula = lambda x: 0.5 * x + 0.3
data_x = np.linspace(0, 1, 100)
plt.plot(data_x, formula(data_x))
plt.show()

In [0]:
outerX = np.random.random(10000)
outerY = 0.5 * outerX + 0.3
outerX, outerY

In [0]:
tf.reset_default_graph()
# shutil.rmtree('tf_logs')
model_dir = "tf_logs/"

In [0]:
n_epoch = 200
learning_rate = 0.5

In [0]:
# Create placeholder to get vlaue
placeholderX = tf.placeholder(tf.float32, shape=[None])

# Create placeholder to get vlaue
placeholderY = tf.placeholder(tf.float32, shape=[None])
        
varW = tf.Variable(tf.random_uniform(shape=[1]), tf.float32)
varB = tf.Variable(tf.random_uniform(shape=[1]), tf.float32)
        
  
# 公式 0.5X + 0.3, varW will be close to 0.5 and varB will be close to 0.3
varY = varW * placeholderX + varB

# tensorboard weights logs
tf.summary.histogram("varW", varW)
tf.summary.histogram("varB", varB)

# mean square error
loss = tf.losses.mean_squared_error(placeholderY, varY)

# tensorboard weights logs
tf.summary.scalar("loss", loss)

# use GradientDescentOptimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

# Adjusting varW and varB to Minimize loss
trainer = optimizer.minimize(loss)

# Merges all summaries collected in the default graph.
merge = tf.summary.merge_all()


with tf.Session() as sess:
    w = tf.summary.FileWriter(model_dir, sess.graph)
    init = tf.global_variables_initializer()
    sess.run(init)
    for i in range(n_epoch):
        _, w_, b_, merge_ = sess.run([trainer, varW, varB, merge], feed_dict={placeholderX: outerX, placeholderY: outerY})
        w.add_summary(merge_, i)
        if i % 20 == 0:
            print('step: {} varW: {}, varB: {}'.format(i, w_, b_) )

            

## Connect to Tensorboard

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [0]:
LOG_DIR = 'tf_logs/'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR))

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"